#### 1223 Турчин
### ДЗ-4 uplift-моделирования

### Задание

<font color=blue size=4> 
    <hr>
1. скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention<br>
2. поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.<br>
3. сделать разбиение набора данных не тренировочную и тестовую выборки<br>
4. провести uplift-моделирование 3 способами:<br>
  -  одна модель с признаком коммуникации (S-learner)<br>
- модель с трансформацией таргета<br>
- вариант с двумя независимыми моделями<br>
5. в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% 3 моделей<br>
*для модели S-learner построить зависимость таргета (конверсии - поле conversion) от значения uplift:<br>
* сделать прогноз и получить uplift для тестовой выборки<br>
* отсортировать тестовую выборку по uplift по убыванию<br>
* разбить на децили (pandas qcut вам в помощь)<br>
* для каждого дециля посчитать среднюю conversionг<br>

<hr>

In [118]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')



<font color=blue size=4> 
    <hr>
1. скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention<br>

<hr>

In [96]:
df = pd.read_csv('data.csv', sep=',')
df.head(5)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0


<font color=blue size=4> 
    <hr>
2. поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.<br>
<hr>

In [97]:
df.rename(columns={'conversion': "target", 'offer': 'treatment'}, inplace=True)
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0


<font color=blue size=4> 
    <hr>
привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.<br>
<hr>

In [98]:
df.treatment.unique()

array(['Buy One Get One', 'No Offer', 'Discount'], dtype=object)

In [99]:
df.loc[df.treatment == 'No Offer', 'treatment'] = 0
df.loc[df.treatment != 0, 'treatment'] = 1
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0


In [100]:
df.channel.unique()

array(['Phone', 'Web', 'Multichannel'], dtype=object)

In [101]:
df.zip_code.unique()

array(['Surburban', 'Rural', 'Urban'], dtype=object)

In [109]:
# Отметим категорийные признаки
cat_features = ['channel', 'zip_code']

<font color=blue size=4> 
    <hr>
3. сделать разбиение набора данных не тренировочную и тестовую выборки<br>
<hr>

#### Вынем метку взаимодействия из датафрейма

In [104]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['target'], axis=1), df['target'], random_state=42, test_size=0.3)

#### Для модели uplift нам надо три компонента X_train (признаки), y_train (целевое значение) и treat_train (признак взаимодействия).
####    Выдернем из X_train признак взаимодействия - treat_train


In [103]:
X_train.head(4)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment
7517,6,92.45,0,1,Urban,0,Web,0
5343,1,796.97,0,1,Rural,1,Multichannel,1
35537,7,390.14,0,1,Urban,1,Phone,1
63983,2,83.03,0,1,Urban,0,Phone,0


In [105]:
# Выдернем из X_train признак взаимодействия - treat_train
treat_train = X_train.treatment

# Удалим из X_train признак взаимодействия
X_train.drop(['treatment'], axis=1, inplace=True)
X_train.head(4)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel
9656,7,434.35,1,0,Urban,1,Web
63037,1,376.59,1,0,Surburban,0,Multichannel
31405,3,140.34,0,1,Urban,1,Phone
58088,3,150.76,0,1,Surburban,0,Web


In [106]:
# Аналогично для теста
# Выдернем из X_test признак взаимодействия - treat_train
treat_valid = X_test.treatment

# Удалим из X_train признак взаимодействия
X_test.drop(['treatment'], axis=1, inplace=True)
X_test.head(4)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel
33042,9,64.85,1,0,Surburban,1,Web
7209,1,144.12,0,1,Urban,0,Web
24367,1,734.25,0,1,Surburban,1,Phone
19599,4,1321.26,1,0,Surburban,1,Multichannel


<font color=blue size=4> 
    <hr>
4. провести uplift-моделирование 3 способами:<br>
- одна модель с признаком коммуникации (S-learner)<br>
- модель с трансформацией таргета<br>
- вариант с двумя независимыми моделями<br>
<hr>

In [131]:
#  Создадим таблицу для сбора результатов
metrics_df = pd.DataFrame(columns=['model', 'uplift@10%', 'uplift@20%', 'uplift@30%'])

### 4.1 Одна модель с признаком коммуникации (S-learner)

In [132]:
# pip install scikit-uplift catboost

In [133]:
# from sklift.models import SoloModel

sm = SoloModel(CatBoostClassifier(iterations=20, random_state=42, silent=True, cat_features=cat_features))

sm = sm.fit(X_train, y_train, treat_train)

uplift_sm = sm.predict(X_test)

sm_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_valid, strategy='by_group', k=0.1)
sm_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_valid, strategy='by_group', k=0.2)
sm_score_30 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_valid, strategy='by_group', k=0.3)

metrics_df = metrics_df.append({
    'model': type(sm).__name__,
    'uplift@10%': sm_score_10,
    'uplift@20%': sm_score_20,
    'uplift@30%': sm_score_30,
}, ignore_index=True)

metrics_df

,model,uplift@10%,uplift@20%,uplift@30%
0,SoloModel,0.122913,0.100633,0.09265


### 4.2 Модель с трансформацией таргета

In [134]:
# from sklift.models import ClassTransformation

ct = ClassTransformation(CatBoostClassifier(iterations=20, random_state=42, silent=True, cat_features=cat_features))

ct = ct.fit(X_train, y_train, treat_train)

uplift_ct = ct.predict(X_test)

ct_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_valid, strategy='by_group', k=0.1)
ct_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_valid, strategy='by_group', k=0.2)
ct_score_30 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_valid, strategy='by_group', k=0.3)


metrics_df = metrics_df.append({
    'model': type(ct).__name__,
    'uplift@10%': ct_score_10,
    'uplift@20%': ct_score_20,
    'uplift@30%': ct_score_30,
}, ignore_index=True)

metrics_df

,model,uplift@10%,uplift@20%,uplift@30%
0,SoloModel,0.122913,0.100633,0.09265
1,ClassTransformation,0.119108,0.103868,0.087555


### 4.3 Вариант с двумя независимыми моделями

In [135]:
from sklift.models import TwoModels


tm = TwoModels(
    CatBoostClassifier(iterations=20, random_state=42, silent=True, cat_features=cat_features),
    CatBoostClassifier(iterations=20, random_state=42, silent=True, cat_features=cat_features),
    method='vanilla'  # независимые модели
)
tm = tm.fit(
    X_train, y_train, treat_train
)

uplift_tm = tm.predict(X_test)

tm_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_valid, strategy='by_group', k=0.1)
tm_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_valid, strategy='by_group', k=0.2)
tm_score_30 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_valid, strategy='by_group', k=0.3)


metrics_df = metrics_df.append({
    'model': type(tm).__name__,
    'uplift@10%': tm_score_10,
    'uplift@20%': tm_score_20,
    'uplift@30%': tm_score_30,
}, ignore_index=True)

metrics_df

,model,uplift@10%,uplift@20%,uplift@30%
0,SoloModel,0.122913,0.100633,0.09265
1,ClassTransformation,0.119108,0.103868,0.087555
2,TwoModels,0.109515,0.091545,0.078819


<font color=blue size=4> 
    <hr>
*для модели S-learner построить зависимость таргета (конверсии - поле conversion) от значения uplift:<br>
* сделать прогноз и получить uplift для тестовой выборки<br>
* отсортировать тестовую выборку по uplift по убыванию<br>
* разбить на децили (pandas qcut вам в помощь)<br>
* для каждого дециля посчитать среднюю conversionг<br>

<hr>

In [142]:
# Создам модель

sm = SoloModel(
    CatBoostClassifier(iterations=20, random_state=42, silent=True, cat_features=cat_features)
)
sm = sm.fit(X_train, y_train, treat_train)
uplift_sm = sm.predict(X_test)

In [149]:
# Внесем предсказанные значения uplift в наш датафрейм
df_test = X_test.copy()
df_test['pred_uplift'] = uplift_sm
df_test['conversion'] = y_test

# Отсортируем по убыванию
df_test = df_test.sort_values(by='pred_uplift', ascending=False)
df_test.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,pred_uplift,conversion
46363,12,206.35,1,1,Surburban,0,Phone,0.180251,1
16356,2,236.10,1,1,Urban,1,Web,0.172420,0
49746,2,223.52,1,1,Surburban,1,Web,0.172420,0
34504,3,237.01,1,1,Urban,1,Web,0.171937,1
57763,3,243.20,1,1,Surburban,1,Web,0.171937,0


In [151]:
# разбьем на децили и для каждого дециля посчитаем среднюю conversionг

df_test['deciles'] = pd.qcut(df_test['pred_uplift'], q=10, labels=False)
for i in range(0, 10):
    tmp = df_test[df_test['deciles'] == i]
    print(f'Для дециля {i} средняя конверсия равна {tmp["conversion"].mean():.3f}')

Для дециля 0 средняя конверсия равна 0.133
Для дециля 1 средняя конверсия равна 0.106
Для дециля 2 средняя конверсия равна 0.127
Для дециля 3 средняя конверсия равна 0.135
Для дециля 4 средняя конверсия равна 0.126
Для дециля 5 средняя конверсия равна 0.145
Для дециля 6 средняя конверсия равна 0.153
Для дециля 7 средняя конверсия равна 0.154
Для дециля 8 средняя конверсия равна 0.165
Для дециля 9 средняя конверсия равна 0.220
